In [1]:
from bullama.bulario import Bulario, BularioType

bulas_questions = Bulario(BularioType.Questions)
bulas_questions.create_data()

bulas_questions.save_csv("datasets/bulario_questions.csv")

In [2]:
bulas_plain_text = Bulario(BularioType.PlainText)
bulas_plain_text.create_data()

bulas_plain_text.save_csv("datasets/bulario_plain_text.csv")

	Time out - https://www.bulario.com/zovirax/
Erro: https://www.bulario.com/zovirax/


In [9]:
import pandas as pd

bulario_questions_df = pd.read_csv('datasets/bulario_questions.csv', delimiter=';')
bulario_plaintext_df = pd.read_csv('datasets/bulario_plain_text.csv', delimiter=';')

In [10]:
from bullama.gerar_perguntas import GeradorDePerguntasMedicamento

gerador = GeradorDePerguntasMedicamento(bulario_questions_df)

In [11]:
df_treino, df_teste = gerador.dividir_dataframe()

df_treino.info()
df_treino.to_csv('datasets/bulario_questions_treino.csv', sep=';', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7241 entries, 0 to 7240
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Nome      7241 non-null   object
 1   Pergunta  7241 non-null   object
 2   Resposta  7241 non-null   object
dtypes: object(3)
memory usage: 169.8+ KB


In [13]:
df_teste.info()
df_teste.to_csv('datasets/bulario_questions_teste.csv', sep=';', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 724 entries, 3081 to 4741
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Nome      724 non-null    object
 1   Pergunta  724 non-null    object
 2   Resposta  724 non-null    object
dtypes: object(3)
memory usage: 22.6+ KB


In [14]:
def generate_prompt(question, context, answer) -> str:
    return f"""[INST] <<SYS>> Responda apenas o que for pedido, usando o contexto:<</SYS>>    
    Context: {context.strip()}
    Question: {question.strip()}[/INST]
    Answer: {answer.strip()}""".strip()

In [15]:
# Carregar os dataframes
bulario_df = pd.read_csv('datasets/bulario_questions_treino.csv', delimiter=';')
bulario_plaintext_df = pd.read_csv('datasets/bulario_plain_text.csv', delimiter=';')

# Criar uma lista de prompts
prompts = []

for index, row in bulario_df.iterrows():
# Iterar sobre as colunas do dataframe bulario_df
    contexto = bulario_plaintext_df.loc[bulario_plaintext_df['Nome'] == bulario_df['Nome'][index], 'texto'].values[0]

    prompt = generate_prompt(bulario_df['Pergunta'][index], contexto.replace('\n',''), bulario_df['Resposta'][index])
    prompts.append(prompt)

# Criar um novo dataframe com a coluna 'text'
prompts_df = pd.DataFrame({'text': prompts})

# Salvar o dataframe em um arquivo CSV com delimitador ;
prompts_df.to_csv('datasets/bulario_prompts_treino.csv', sep=';', index=False)